In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
import numpy as np
import pymongo

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
# Define database and collection
db = client.jobs_db
db.jobs_glassdoor.drop()
jobs_glassdoor_mongo = db.jobs_glassdoor

In [4]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [5]:
job_dict = {}

In [6]:
job_categories = ["Data Analyst", "Data Scientist"]
region = "United States"
posted_time = "7"
for job_category in job_categories:
    
    glassdoor_url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword=' + job_category + \
    "&locT=N&locId=1&locKeyword=" + region + "&jobType=all&fromAge=" + posted_time + \
    "&minSalary=0&includeNoSalaryJobs=true&radius=-1&cityId=-1&minRating=0.0&industryId=-1" +\
    "&companyId=-1&applicationType=0&employerSizes=0&remoteWorkType=0"

    browser.visit(glassdoor_url)
    job_counter = 1
    for x in range(1, 4):
        print("Page no.:", x, "--------")
        
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')

        job_section = soup.find('ul', class_='jlGrid')
        job_list = job_section.find_all('li', class_='jl')

        for job in job_list:
            # Job Title
            job_title = job.find('div', class_='jobTitle').a.text            
            # Company Name
            company_text = job.find('div', class_='empLoc').text.split("–")
            company_name = company_text[0][1:][:-1] # formatting to remove the spaces in beginning and end
            # City and State
            location = job.find('div', class_='empLoc').span.text.split(", ")          
            # Salary information - Min, Max and Avg
            try:
                salary_text = job.find('span', class_="green").text.replace("(Glassdoor Est.)", "").replace("(Employer Est.)", "")
                sal_range = salary_text[1:].split("-")
                min_sal = int(sal_range[0].lstrip("$").rstrip("k"))
                max_sal = int(sal_range[1].lstrip("$").rstrip("k"))
                avg_sal = (max_sal+min_sal)/2   
            except AttributeError:
                min_sal = None
                max_sal = None
                avg_sal = None                
                
            print('job no.:', job_counter, '-------------')
            job_counter +=1

            job_dict = {"Title": job_title,
                        "Company": company_name,
                        "City": location[0],
                        "State": location[-1],
                        "Designation": job_category,
                        "Min Salary": min_sal,
                        "Max Salary": max_sal,
                        "Avg Salary": avg_sal,
                        "Source": "Glassdoor"
                        }
            jobs_glassdoor_mongo.insert_one(job_dict)
            
        browser.click_link_by_partial_text('Next')
        try:
            #If a pop-up window opens, close    
            browser.find_by_css('.xBtn').click()
        except:
            pass
        sleep(np.random.choice(range(4,7)))
    
    sleep(10)
    

Page no.: 1 --------
job no.: 1 -------------
job no.: 2 -------------
job no.: 3 -------------
job no.: 4 -------------
job no.: 5 -------------
job no.: 6 -------------
job no.: 7 -------------
job no.: 8 -------------
job no.: 9 -------------
job no.: 10 -------------
job no.: 11 -------------
job no.: 12 -------------
job no.: 13 -------------
job no.: 14 -------------
job no.: 15 -------------
job no.: 16 -------------
job no.: 17 -------------
job no.: 18 -------------
job no.: 19 -------------
job no.: 20 -------------
job no.: 21 -------------
job no.: 22 -------------
job no.: 23 -------------
job no.: 24 -------------
job no.: 25 -------------
job no.: 26 -------------
job no.: 27 -------------
job no.: 28 -------------
job no.: 29 -------------
job no.: 30 -------------
job no.: 31 -------------
Page no.: 2 --------
job no.: 32 -------------
job no.: 33 -------------
job no.: 34 -------------
job no.: 35 -------------
job no.: 36 -------------
job no.: 37 -------------
job n

In [7]:
browser.quit()

In [8]:
jobs_glassdoor_listing = db.jobs_glassdoor.find()
for listing in jobs_glassdoor_listing:
    print(listing)

{'_id': ObjectId('5c083ba50ea05e36e02577d6'), 'Title': 'Data Analyst', 'Company': 'T. Rowe Price', 'City': 'Baltimore', 'State': 'MD', 'Designation': 'Data Analyst', 'Min Salary': 69, 'Max Salary': 90, 'Avg Salary': 79.5, 'Source': 'Glassdoor'}
{'_id': ObjectId('5c083ba50ea05e36e02577d7'), 'Title': 'Data Analyst', 'Company': 'Bank of America', 'City': 'Plano', 'State': 'TX', 'Designation': 'Data Analyst', 'Min Salary': 68, 'Max Salary': 91, 'Avg Salary': 79.5, 'Source': 'Glassdoor'}
{'_id': ObjectId('5c083ba50ea05e36e02577d8'), 'Title': 'Analyst', 'Company': 'Freeman', 'City': 'Dallas', 'State': 'TX', 'Designation': 'Data Analyst', 'Min Salary': 46, 'Max Salary': 73, 'Avg Salary': 59.5, 'Source': 'Glassdoor'}
{'_id': ObjectId('5c083ba50ea05e36e02577d9'), 'Title': 'Data Analyst', 'Company': 'SMC 3', 'City': 'Peachtree City', 'State': 'GA', 'Designation': 'Data Analyst', 'Min Salary': 53, 'Max Salary': 72, 'Avg Salary': 62.5, 'Source': 'Glassdoor'}
{'_id': ObjectId('5c083ba50ea05e36e0257